In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate=3e-4
eval_iters = 250
dropout = 0.2

cuda


In [32]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [33]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [34]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[58, 69,  1, 56, 71, 54, 56, 64],
        [73,  1, 64, 62, 67, 57,  1, 68],
        [60, 72,  1, 66, 54, 67, 78,  0],
        [57,  1, 72, 56, 54, 71, 56, 58]], device='cuda:0')
targets:
tensor([[69,  1, 56, 71, 54, 56, 64,  1],
        [ 1, 64, 62, 67, 57,  1, 68, 59],
        [72,  1, 66, 54, 67, 78,  0, 57],
        [ 1, 72, 56, 54, 71, 56, 58, 65]], device='cuda:0')


In [35]:
@torch.no_grad()
def estime_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [36]:

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [37]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
 
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :]
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


?4h6,o*LfmPLKg
z_J]d*VjA.?_[ciY"m(vBf.[
L3ZmAkinVOX1_e6x1-hHWZzs])cjwqur
bi y6FmWhSXfrPK4mMRnMzIlQF6ppvw5QlW*Z8E"aT,t"x5Qa&P3KNqsAdsz_X.j!MCTn[r p3pt
&XRrLp2c;NA)﻿2cDH,[Y
eoq9dK *["48Y -gFWLxv,eO:D-8(ENIGa;2oqdC9mn*W"x﻿:[JxIgVFI2Vmn1_g﻿gFmk
ou8:qM0p_L'[6Bt.n7W8YuZC9w﻿*!ad7MS'01!MsJ
hH7Vmpa'!Mzo_EzArYTc*tljmsALWY8hxON(b.awCWfo-p_T3nX:K p3r
!T5335Qq9Fr(,g﻿6bCgGl(k7"Y,94XUN"A3GEq?ITPQ*[-p_EUhE;4USfTnkw'r
yxJzj
T?[RucZ*:'ewC]:QmLIZX.o [(I:spDHsAM3﻿XatUVt7t!hF6A﻿JxJst8j:?L7﻿J FY]HEzoVDZ[K,[!ZbHhO:Q_g


In [42]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estime_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.529, val loss: 4.521
step: 250, train loss: 4.455, val loss: 4.464
step: 500, train loss: 4.408, val loss: 4.409
step: 750, train loss: 4.339, val loss: 4.369
4.364394664764404


In [40]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


B
BgS
[]"41w5rl3qvNIlPe4Ep﻿"SGW?--ZngKCs:D6B ﻿602ZpDbzx-!Ms56dv3:V;)onNFPsA8&W*aG]qgfzoXUCO21-zuxk1wA)Ozz7rYL7!sZr"_Nw5Rn-Cu01!X4g1[D[Y,GqkT.Q7e6f.7!IkFmk7﻿8ngHf)sAwWSxII2inj4-9b-] gout5I']MwitVnk6sJce&
vLongquh;'b)5X4gFkM-peeQmw1XL-RdatN)clp0QOOa;muhQVttwn2mJnDtA3peRvHpIiX60Q1UfQ(;M0pWR9udsK bdyif2Z&,[BatRj
7W*ZTUV)k:6!A7Te.2r
3*Z&sJ5D3Hp_4-?2CG'[5nMx﻿GUY6]m78oX'am*MtHfouHkRv8Z6] x((?L&!*.cMOM32q*FZldk:XOnE"﻿6Boc6C*6uZ!Tgd9] "ondE[Twt"2cG'L Qpeyx7kN3H'QE;Sp
wp[
3r9uovr'-dscWvwn0TBdQhqfB3QR'gi7﻿
